# Using Together AI with Composio to Send Emails

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github.com/togethercomputer/together-cookbook/blob/main/Agents/Composio/Agents_Composio.ipynb)

<img src="../../images/composio.png" width="700">

This notebook demonstrates how to integrate Together AI's language models with Composio's tools to create an AI agent that can send emails. We'll show:

1. Setting up the required packages and authentication
2. Configuring the email tool with Composio
3. Creating a basic email-sending agent
4. Adding preprocessing to customize email behavior
5. Executing and handling the email responses

Let's see how we can combine these powerful tools to automate email communication!

## Prerequisites
- Together AI API key - see here here https://api.together.ai/
- Composio API key - see here https://app.composio.dev/developers
- Gmail account connected to Composio - https://app.composio.dev/integrations - You can also watch this video to see how [integrations with Composio work.](https://www.youtube.com/watch?v=LmyWy4LiedQ)

In [4]:
# install the required packages
!pip install -qU together composio-togetherai

## Gmail Configuration
We need to connect our Gmail account to Composio. This will open a browser window for authentication.

In [ ]:
# You can add the gmail tool by running the following command or manually going to https://app.composio.dev/integrations
!composio login
!composio add gmail

## Basic Email Tool Call

Here we create a simple integration between Together AI's LLM and Composio's Gmail tool:
1. Initialize Together AI client
2. Create Composio toolset
3. Configure email sending capability
4. Make an LLM call with the email tool to see if everything is working

In [1]:
import os
from composio_togetherai import ComposioToolSet, Action
from together import Together

# Initialize Together AI and Composio clients with API keys
client = Together(api_key=os.getenv("TOGETHER_API_KEY"))
toolset = ComposioToolSet(api_key=os.getenv("COMPOSIO_API_KEY"))

# Get the Gmail send email tool
send_email_tool = toolset.get_tools(
    [Action.GMAIL_SEND_EMAIL], check_connected_accounts=False
)

# Create a chat completion with email capability
response = client.chat.completions.create(
    model="Qwen/Qwen2.5-72B-Instruct-Turbo",
    messages=[
        {"role": "system", "content": "You are Alex, a product manager at an AI company."},
        {
            "role": "user",
            "content": "Send an email to John, who is an ML engineer on the team, to inquire about a good time to meet next week to review the upcoming launch.",
        },
    ],
    tools=send_email_tool,
)

response.model_dump()

/opt/anaconda3/envs/cookbook/lib/python3.12/site-packages/composio/tools/toolset.py:543: UserWarning: Not verifying connected accounts for apps. Actions may fail when the Agent tries to use them.
  warnings.warn(


{'id': 'nnSQh4S-4yUbBN-925f455aa84c36d8',
 'object': <ObjectType.ChatCompletion: 'chat.completion'>,
 'created': 1742914753,
 'model': 'Qwen/Qwen2.5-72B-Instruct-Turbo',
 'choices': [{'index': 0,
   'logprobs': None,
   'seed': 15668879311493286000,
   'finish_reason': <FinishReason.ToolCalls: 'tool_calls'>,
   'message': {'role': <MessageRole.ASSISTANT: 'assistant'>,
    'content': None,
    'tool_calls': [{'id': 'call_4rx0ntnagtqyrwml4fwvmhz9',
      'type': 'function',
      'function': {'name': 'GMAIL_SEND_EMAIL',
       'arguments': '{"user_id":"me","recipient_email":"john@mlteam.com","subject":"Meeting Request: Review Upcoming Launch","body":"Hi John,\\n\\nI hope this message finds you well. Could you please let me know a good time for us to meet next week to review the upcoming launch? Your availability is much appreciated.\\n\\nBest regards,\\nAlex","is_html":false}'},
      'index': 0}]}}],
 'prompt': [],
 'usage': {'prompt_tokens': 535,
  'completion_tokens': 104,
  'total_to

## Modify Gmail tool schema

We can customize the email behavior using preprocessors and schema processors:
- Preprocessor: Modifies the input before sending (e.g., changing recipient email)
- Schema Processor: Modifies the tool schema (e.g., removing fields from LLM's view)

In [2]:
# Preprocessor to override recipient email
def gmail_preprocessor(inputs: dict) -> dict:
    inputs["recipient_email"] = "my_email.dev"  # Change to an email you can access to test!
    return inputs

# Schema processor to hide recipient field from LLM
def gmail_schema_processor(schema: dict) -> dict:
    del schema["recipient_email"]
    return schema

# Create processed email tool with both processors
processed_send_email_tool = toolset.get_tools(
    actions=[Action.GMAIL_SEND_EMAIL],
    processors={
        "schema": {Action.GMAIL_SEND_EMAIL: gmail_schema_processor},
        "pre": {Action.GMAIL_SEND_EMAIL: gmail_preprocessor},
    },
    check_connected_accounts=True,
)

## Executing the Email Send

Now we'll execute the email send with our processed tool and handle the response

In [3]:
# Execute chat completion with processed tool
response = client.chat.completions.create(
    model="Qwen/Qwen2.5-72B-Instruct-Turbo",
    messages=[
        {"role": "system", "content": "You are Alex, a product manager at an AI company."},
        {
            "role": "user",
            "content": "Send an email to John, who is an ML engineer on the team, to inquire about a good time to meet next week to review the upcoming launch.",
        },
    ],
    tools=processed_send_email_tool,
)

# Handle the tool calls and send the email
exec_response = toolset.handle_tool_calls(response)


In [4]:
# We can see the email was sent successfully!
exec_response

[{'data': {'response_data': {'id': '195cdd11b381f385',
    'threadId': '195cdd11b381f385',
    'labelIds': ['UNREAD', 'SENT', 'INBOX']}},
  'error': None,
  'successfull': True,
  'successful': True}]

<img src="../../images/email.png" width="300">